In [1]:
import pandas as pd
import numpy as np
import scipy as scp
from bs4 import BeautifulSoup
import requests as rq
import re
from requests.exceptions import HTTPError
import urllib
import urllib.request

from selenium import webdriver
from reppy.robots import Robots
from urllib.parse import urlparse
import time
import operator

## MtgMintCards

In [2]:
urls = ['http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/the-great-henge',
       'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/oko-thief-of-crowns',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/arcane-signet-brawl-deck',
       'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/questing-beast',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/brazen-borrower',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/knights-charge-brawl-deck',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/shimmer-dragon-brawl-deck',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/steelbane-hydra-brawl-deck',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/murderous-rider',
      'http://www.mtgmintcard.com/mtg/singles/eld/eng-reg/once-upon-a-time']
infoMtgmintCard = []

In [3]:
def ExtractMtgMintCards(url):
    
    codigo_fonte = rq.get(url,headers={'content-type': 'text/html'})
    texto = codigo_fonte.text
    soup = BeautifulSoup(texto,'html.parser')
    
    # InfoBox
    root_div = soup.find('div',itemtype="http://schema.org/Product")
    
    #Nome
    Nome = root_div.find('span',itemprop='name').text
    
    #Preço
    preço = '$ ' + root_div.find('span',{'itemprop':'price'}).text
    
    #Nome do Set
    Nome_set = root_div.find('span',content = True).text
    
    #Casting Cost 
    aux_CastingCost = root_div.find('strong',text = 'Casting Cost')
    p_CastingCost = aux_CastingCost.parent
    Lista_Img_castingcost = p_CastingCost.findAll('img',alt=True)
    CastingCost = []
    for image in Lista_Img_castingcost:
        CastingCost.append(image['alt'])    
    
    #Descrição
    aux_descrição = root_div.find('span',{'itemprop':'description'}).text
    descricao = re.sub('[\s+]', ' ', aux_descrição)
    #regex = '(<img)(.*)(\/>\.)'
    #imagens = re.compile(regex)          
    
    #Raridade 
    aux_Raridade = root_div.find('strong',text = 'Rarity')
    Raridade = aux_Raridade.parent.text
    
    return infoMtgmintCard.append(dict(zip(['URL','Nome-Carta','Set','Raridade','Casting Cost','Preço','Descrição'],[url,Nome,Nome_set,Raridade,CastingCost,preço,descricao])))


In [4]:
for url in urls:
    ExtractMtgMintCards(url)

In [5]:
data = pd.DataFrame(infoMtgmintCard,columns=['URL','Nome-Carta','Set','Raridade','Casting Cost','Preço','Descrição'])
data

,URL,Nome-Carta,Set,Raridade,Casting Cost,Preço,Descrição
0,http://www.mtgmintcard.com/mtg/singles/eld/eng...,The Great Henge,Throne of Eldraine,Rarity Mythic,"[7, Green, Green]",$ 10.99,"This spell costs less to cast, where X is the..."
1,http://www.mtgmintcard.com/mtg/singles/eld/eng...,"Oko, Thief of Crowns",Throne of Eldraine,Rarity Mythic,"[1, Green, Blue]",$ 37.99,2: Create a Food token. 1: Target artifact o...
2,http://www.mtgmintcard.com/mtg/singles/eld/eng...,Arcane Signet (Brawl Deck),Throne of Eldraine,Rarity Common,[2],$ 10.49,: Add one mana of any color in your commander'...
3,http://www.mtgmintcard.com/mtg/singles/eld/eng...,Questing Beast,Throne of Eldraine,Rarity Mythic,"[2, Green, Green]",$ 35.99,"Vigilance, deathtouch, haste Questing Beast ca..."
4,http://www.mtgmintcard.com/mtg/singles/eld/eng...,Brazen Borrower,Throne of Eldraine,Rarity Mythic,"[1, Blue, Blue]",$ 15.99,Flash Flying Brazen Borrower can block only cr...
5,http://www.mtgmintcard.com/mtg/singles/eld/eng...,Knights' Charge (Brawl Deck),Throne of Eldraine,Rarity Rare,"[1, White, Black]",$ 4.99,"Whenever a Knight you control attacks, each op..."
6,http://www.mtgmintcard.com/mtg/singles/eld/eng...,Shimmer Dragon (Brawl Deck),Throne of Eldraine,Rarity Rare,"[4, Blue, Blue]",$ 5.49,Flying As long as you control four or more art...
7,http://www.mtgmintcard.com/mtg/singles/eld/eng...,Steelbane Hydra (Brawl Deck),Throne of Eldraine,Rarity Rare,"[X, Green, Green]",$ 4.99,Steelbane Hydra enters the battlefield with X ...
8,http://www.mtgmintcard.com/mtg/singles/eld/eng...,Murderous Rider,Throne of Eldraine,Rarity Rare,"[1, Black, Black]",$ 6.99,"Lifelink When Murderous Rider dies, put it on ..."
9,http://www.mtgmintcard.com/mtg/singles/eld/eng...,Once Upon a Time,Throne of Eldraine,Rarity Rare,"[1, Green]",$ 19.99,If this spell is the first spell you've cast t...


## Scryfall

In [6]:
urlsScryfall = ['https://scryfall.com/card/c17/2/balan-wandering-knight','https://scryfall.com/card/ddu/1/ezuri-renegade-leader',
               'https://scryfall.com/card/a25/3/akromas-vengeance','https://scryfall.com/card/txln/2/illusion',
                'https://scryfall.com/card/c17/1/alms-collector','https://scryfall.com/card/hou/158/abandoned-sarcophagus',
               'https://scryfall.com/card/ori/127/abbot-of-keral-keep','https://scryfall.com/card/rna/151/absorb',
               'https://scryfall.com/card/hou/158/abandoned-sarcophagus','https://scryfall.com/card/wwk/20/stoneforge-mystic']
infoScryfall = []

In [7]:
def ExtractScryfall(url):
    codigo_fonte = rq.get(url,headers={'content-type': 'text/html'})
    texto = codigo_fonte.text
    soup = BeautifulSoup(texto,'html.parser')
    
    #Scryfall raiz da carta
    root_div = soup.find('div',class_="inner-flex")
    #print(root_div)
    
    # texto-carta
    carta_texto = root_div.find('div',class_='card-text')
    
    #Casting cost
    auxCastingCost = carta_texto.find('span',class_='card-text-mana-cost')
    if auxCastingCost == None:
        CastingCost = None
    else:
        abbrCastingCost = auxCastingCost.findAll('abbr')
        CastingCost = []
        for abbr in abbrCastingCost:
            simbolo = str(abbr.text)
            texto = str(abbr['title'])
            FinalTexto = simbolo +' '+texto
            CastingCost.append(FinalTexto)
    #print(CastingCost)
    
    # Descrição
    oracleTexto = carta_texto.find('div',class_="card-text-oracle").text
    flavorTexto = carta_texto.find('div',class_="card-text-flavor")
    if flavorTexto == None:
        descricao = oracleTexto
        descricao = re.sub('[\s+]', ' ', descricao)
    else:
        descricao = oracleTexto + '\n' + flavorTexto.text
        descricao = re.sub('[\s+]', ' ', descricao)

    #print(descricao)
    
    
    #Detalhes da edição,preço raridade
    auxDetalhes = root_div.find('div',class_='prints')
    
    # Set
    Set = auxDetalhes.find('span',class_="prints-current-set-name").text
    Set = re.sub('[\s+]', ' ', Set)
    #print(Set)
    
    # Raridade 
    Raridade = auxDetalhes.find('span',class_="prints-current-set-details").text
    Raridade = re.sub('[\s+]', '', Raridade)
   # print(Raridade)
    
    # Preço
    Preco = auxDetalhes.find('a',class_="currency-usd").text
   # print(Preco)
    
    
    # Nome
    # Usar um auxiliar para eliminar tags desnecessarias para retirar o nome.
    aux_carta_texto = root_div.find('div',class_='card-text')
    codetags = aux_carta_texto.findAll('span')
    for codetag in codetags:
        codetag.extract()
    
    Nome = aux_carta_texto.find('h1',class_="card-text-title").text
    Nome = re.sub('[\s+]', ' ', Nome)
    #print(Nome)
    
    
    return infoScryfall.append(dict(zip(['URL','Nome-Carta','Set','Raridade','Casting Cost','Preço','Descrição'],
                                        [url,Nome,Set,Raridade,CastingCost,Preco,descricao])))


In [8]:
for url in urlsScryfall:
    ExtractScryfall(url)

In [9]:
dataScryfall = pd.DataFrame(infoScryfall,columns=['URL','Nome-Carta','Set','Raridade','Casting Cost','Preço','Descrição'])
dataScryfall

,URL,Nome-Carta,Set,Raridade,Casting Cost,Preço,Descrição
0,https://scryfall.com/card/c17/2/balan-wanderin...,"Balan, Wandering Knight",Commander 2017 (C17),#2·Rare·English,"[{2} two generic mana, {W} one white mana, {W}...",$8.67,"First strike Balan, Wandering Knight has doub..."
1,https://scryfall.com/card/ddu/1/ezuri-renegade...,"Ezuri, Renegade Leader",Duel Decks: Elves vs. Inventors (DDU)...,#1·MythicRare·English,"[{1} one generic mana, {G} one green mana, {G}...",✶ $1.89,{G}: Regenerate another target Elf. {2}{G}{G}...
2,https://scryfall.com/card/a25/3/akromas-vengeance,Akroma's Vengeance,Masters 25 (A25),#3·Rare·English,"[{4} four generic mana, {W} one white mana, {W...",$0.45,"Destroy all artifacts, creatures, and enchant..."
3,https://scryfall.com/card/txln/2/illusion,Illusion,Ixalan Tokens (TXLN),#2·Common·English,None,$0.10,When this creature becomes the target of a sp...
4,https://scryfall.com/card/c17/1/alms-collector,Alms Collector,Commander 2017 (C17),#1·Rare·English,"[{3} three generic mana, {W} one white mana]",$2.37,Flash If an opponent would draw two or more c...
5,https://scryfall.com/card/hou/158/abandoned-sa...,Abandoned Sarcophagus,Hour of Devastation (HOU),#158·Rare·English,[{3} three generic mana],✶ $0.79,You may cast nonland cards with cycling from ...
6,https://scryfall.com/card/ori/127/abbot-of-ker...,Abbot of Keral Keep,Magic Origins (ORI),#127·Rare·English,"[{1} one generic mana, {R} one red mana]",✶ $0.98,Prowess (Whenever you cast a noncreature spel...
7,https://scryfall.com/card/rna/151/absorb,Absorb,Ravnica Allegiance (RNA),#151·Rare·English,"[{W} one white mana, {U} one blue mana, {U} on...",✶ $2.87,Counter target spell. You gain 3 life. “In ...
8,https://scryfall.com/card/hou/158/abandoned-sa...,Abandoned Sarcophagus,Hour of Devastation (HOU),#158·Rare·English,[{3} three generic mana],✶ $0.79,You may cast nonland cards with cycling from ...
9,https://scryfall.com/card/wwk/20/stoneforge-my...,Stoneforge Mystic,Worldwake (WWK),#20·Rare·English,"[{1} one generic mana, {W} one white mana]",✶ $53.07,When Stoneforge Mystic enters the battlefield...


## Abugames

In [10]:
urlsAbu = ['https://abugames.com/magic-the-gathering/singles/product-detail/Aggressive-Urge-%252D-FOIL/10th-Edition/English/',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Abundance/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Abundance-%252D-FOIL/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Academy-Researchers/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Academy-Researchers-%252D-FOIL/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Adarkar-Wastes/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Afflict/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Afflict-%252D-FOIL/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Agonizing-Memories/10th-Edition/English',
          'https://abugames.com/magic-the-gathering/singles/product-detail/Flowstone-Salamander/Tempest/English/']
infoAbu = []

In [141]:
def ExtractAbu(url):
    
    # Selenium
    driver = webdriver.Chrome("Driver\\chromedriver78\\chromedriver.exe")
    driver.get(url)
    texto = driver.execute_script("return document.documentElement.outerHTML")
    driver.quit
    
    # Parsing
    soup = BeautifulSoup(texto,'html.parser')
    
    # Tag Informação central
    root_div = soup.find('div',class_="singleCardMain")
    #print(root_div)
    
    # Detalhes
    infoDetalhes = root_div.find('div',class_='row first')
    #print(infoDetalhes)
    
    # Nome
    Nome = infoDetalhes.find('h1').text
    print(Nome)
    
    # Set
    Set = infoDetalhes.find('div',class_="col-md-2 col-sm-2 col-xs-4").text
    print(Set)
    
    # Idioma
    auxIdioma = infoDetalhes.find('div',class_="col-md-1 col-sm-1 col-xs-2") 
    
    imgIdioma = auxIdioma.findAll('img',src=True)
    Imagens = []
    for image in imgIdioma:
        Imagens.append(image['src'])
        
    regexIdioma =  re.compile('([A-Z]\w+)')    
    srcImgIdioma = re.search(regexIdioma,Imagens[0])
    
    if srcImgIdioma:
        Idioma = Imagens[0][srcImgIdioma.start():srcImgIdioma.end()]
    else:
        Idioma = None
    
    print(Idioma)
    
    # Raridade
    # o Idioma e a raridade tem a msm classe
    auxRaridade = infoDetalhes.findAll('div',class_="col-md-1 col-sm-1 col-xs-2")
    Raridade = auxRaridade[1].text
    print(Raridade)
    
    
    # Descrição 
    originalTexto = root_div.find("div",{"id":"originalTextundefined"}).text
    flavorTexto = root_div.find("div",{"id":"FlavorTextundefined"}).text
    
    Descrição = originalTexto +"\n"+flavorTexto
    print(Descrição)
    
    #Preço
    Preco = root_div.find('div',class_="col-md-1 col-xs-2 ng-star-inserted").text
    print(Preco)
    print('-------------------------------------------------------------------------------------------------------------------')
    #Casting cost
    auxCastingcost = infoDetalhes.find('div',class_="col-md-2 col-sm-2 col-xs-3")
    tags_i = auxCastingcost.findAll('i')
    print(tags_i)
    
    return root_div

In [142]:
ExtractAbu(urlsAbu[0])

 Aggressive Urge - FOIL 
 10th Edition 
English
Common
Target creature gets +1/+1 until end of turn.
Draw a card.
The power of the wild, concentrated in a single charge.
$0.89
-------------------------------------------------------------------------------------------------------------------

[<i _ngcontent-nar-c5="" class="ms ms-1 ms-cost ms-shadow ng-star-inserted"></i>, <i _ngcontent-nar-c5="" class="ms ms-g ms-cost ms-shadow ng-star-inserted"></i>]


<div class="singleCardMain"><div class="tableHere"><div class="row heading"><div class="col-md-3">Name</div><div class="col-md-2">EDITION</div><div class="col-md-1">LANGUAGE</div><div class="col-md-2">MANA</div><div class="col-md-2">TYPE</div><div class="col-md-1">P / T (L)</div><div class="col-md-1">RARITY</div></div></div><app-single-detail-card _nghost-nar-c5=""><!-- --><div _ngcontent-nar-c5="" class="productHere ng-star-inserted"><!-- --><!-- --><div _ngcontent-nar-c5="" class="tableHere ng-star-inserted"><div _ngcontent-nar-c5="" class="row first"><div _ngcontent-nar-c5="" class="col-md-3 col-sm-3 col-xs-6"><!-- --><i _ngcontent-nar-c5="" aria-hidden="true" class="icon fa fa-camera spoiler my-gallery294699-open ng-star-inserted" style="cursor: pointer"></i><!-- --><h1 _ngcontent-nar-c5=""> Aggressive Urge - FOIL <!-- --></h1></div><div _ngcontent-nar-c5="" class="col-md-2 col-sm-2 col-xs-4"><i _ngcontent-nar-c5="" class="ss ss-fw ss-align-edition ss-10e"></i> 10th Edition </div><

In [222]:
for url in urlsAbu:
    extractor_Abu(url)


In [223]:
dataAbu = pd.DataFrame(infoAbu,columns=['Titulo','Set','Raridade','Descrição','Tipo'])
dataAbu

,Titulo,Set,Raridade,Descrição,Tipo
0,Aggressive Urge - FOIL,10th Edition,ommon,Target creature gets +1/+1 until end of turn.\...,FOIL10thEditio
1,Aggressive Urge - FOIL,10th Edition,ommon,Target creature gets +1/+1 until end of turn.\...,"[FOIL, 10th, Edition, Instant]"
2,Abundance,10th Edition,are,"If you would draw a card, you may instead choo...",[Enchantment]
3,Abundance - FOIL,10th Edition,are,"If you would draw a card, you may instead choo...","[10th, Edition, Enchantment]"
4,Academy Researchers,10th Edition,Uncommon,"When Academy Researchers comes into play, you ...","[Edition, Creature, -, Human, Wizard2, /]"
5,Academy Researchers - FOIL,10th Edition,Uncommon,"When Academy Researchers comes into play, you ...","[FOIL, 10th, Edition, Creature, -, Human, Wiza..."
6,Adarkar Wastes,10th Edition,are,: Add to your mana pool.\n: Add or to your ...,"[Edition, Land]"
7,Afflict,10th Edition,ommon,Target creature gets -1/-1 until end of turn.\...,[Instant]
8,Afflict - FOIL,10th Edition,ommon,Target creature gets -1/-1 until end of turn.\...,"[10th, Edition, Instant]"
9,Agonizing Memories,10th Edition,ncommon,Look at target player's hand and choose two ca...,"[Edition, Sorcery]"
